In [ ]:
# Set the Seed Value for the Notebook so the Results are Reproducible
from numpy.random import seed
seed(1)

In [ ]:
# Import Modules & Dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import tensorflow
import pandas as pd

In [ ]:
# Import Modules & Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Read in CSV File of Survey Results with an Adjusted 1-3 Likert Scale
df = pd.read_csv("Data/personality_traits_adjusted_scale_MOSHER.csv")

#Drop the Null Columns Where All Values Are Null
df = df.dropna(axis='columns', how='all')

#Drop the Null Rows
df = df.dropna()
df.head()

In [ ]:
# Adjusted 1-3 Likert Scale Features
adj_heights_features = df[["Funniness", "Loneliness", "Energy_levels", "Knowing_the_right_people", "Eating_to_survive",
                           "Waiting", "Life_struggles", "Appearence_and_gestures", "Self_criticism", "Health"]]

adj_snakes_features = df[["Health", "Public_speaking", "Thinking_ahead", "Getting_angry", "God", "Number_of_friends",
                          "Children"]]

adj_spiders_features = df[["Number_of_friends", "Children", "Life_struggles", "Happiness_in_life", "Public_speaking", 
                           "Questionnaires_or_polls", "Getting_angry", "Appearence_and_gestures", "Getting_up",
                           "Unpopularity"]]

In [ ]:
# Define the Features (X) and Target (y) to be Used in the Model
X = adj_heights_features
y = df["Heights"]

# X = adj_snakes_features
# y = df["Snakes"]

# X = adj_spiders_features
# y = df["Spiders"]

print(X.shape, y.shape)

In [ ]:
# Use Train_Test_Split to Create Training and Testing Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [ ]:
# Create a MinMaxScaler Model and Fit it to the Training Data
X_scaler = MinMaxScaler().fit(X_train)

# Transform the Training and Testing Data Using the X_Scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Label-Encode Dependent/Target Variables For Training and Testing Data
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
# Convert Encoded Labels Using One-Hot-Encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
# Create a Normal Neural Network Model and Add Input and Output Layers
model = Sequential()
model.add(Dense(units=70, activation='relu', input_dim= (X.shape[1])))
model.add(Dense(units=3, activation='softmax'))

In [ ]:
# Compile and Fit the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Review Model Layers
model.summary()

In [ ]:
# Fit Model and Set Final Parameters Before Running on Training Data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# Evaluate Model Accuracy and Loss For Test Data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Compare Predicted Values to Actual Values For Each Survey Respondant
predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(predictions)
pd.DataFrame({"Prediction": prediction_labels, "Actual": y_test}).head(50)

In [ ]:
# Save Each Model Once the Optimal Set of Parameters is Established For Each Model
# model.save("models/heights_model_trained.h5")
# model.save("models/snakes_model_trained.h5")
# model.save("public_speaking_model_trained.h5")
# model.save("ageing_model_trained.h5")
# model.save("models/spiders_model_trained.h5")